### Dog Breed Identification
1. Will use a Resnet-50 model(Difference between Resnet-34 and Resnet-50?)
2. Will be using Tensorflow and Keras
3. Converting images to greyscale

In [1]:
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image    
from keras.models import Sequential
from keras.layers import Dense, Activation, GlobalAveragePooling2D
from tqdm import tqdm
from matplotlib import pyplot as plt
import cv2
import sys
import os
import numpy as np

Using TensorFlow backend.


Resnet50 (Residual NN) is a pre-trained model using the ImageNet dataset which can classify an object from one of 1000 categories

In [2]:
# define ResNet50 model
ResNet50_model = ResNet50(weights='imagenet')

1. path_to_tensor reads image paths, converts to images and resizes it to a square image that is 224x224 pixels, and returns a 4D array (referred to as a ‘tensor’) suitable for supplying to a Keras CNN

In [3]:
def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(224, 224))
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

1. Download data from kaggle. Train and Test

In [8]:
pth = "/Users/mgadgil/Documents/dog_breed_classification/data/train"
train_paths = []
for root, dirs, files in os.walk(pth):
    for file in files:
        train_paths.append(os.path.join(root,file))
train_data = paths_to_tensor(train_paths)

100%|██████████| 10222/10222 [01:33<00:00, 109.73it/s]


All pre-trained models have the additional normalization step that the mean pixel must be subtracted from every pixel in each image. This is implemented in the imported function preprocess_input

ResNet50_predict_labels returns an integer corresponding to the model’s predicted object class by taking the argmax of the predicted probability vector, which we can identify with an object category through the use of the ImageNet labels dictionary

In [9]:
from keras.applications.resnet50 import preprocess_input, decode_predictions

def ResNet50_predict_labels(img_path):
    # returns prediction vector for image located at img_path
    img = preprocess_input(path_to_tensor(img_path))
    return np.argmax(ResNet50_model.predict(img))

### returns "True" if a dog is detected in the image stored at img_path
def dog_detector(img_path):
    prediction = ResNet50_predict_labels(img_path)
    return ((prediction <= 268) & (prediction >= 151)) 

#### Creating a CNN that classifies dog breeds


Use of Transfer Learning - a method that allows us to use Networks that have been pre-trained on a large dataset. By keeping the early layers and only training newly added layers.

The last convolutional output of ResNet-50 is fed as input to our model. We only add a Global Average Pooling layer and a Fully Connected layer, where the latter contains one node for each dog category and has a Softmax activation function.

In [10]:
### Define your architecture.
Resnet50_model = Sequential()
Resnet50_model.add(GlobalAveragePooling2D(input_shape=train_data.shape[1:]))
Resnet50_model.add(Dense(120, activation='softmax')) # 
Resnet50_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
global_average_pooling2d_1 ( (None, 3)                 0         
_________________________________________________________________
dense_1 (Dense)              (None, 120)               480       
Total params: 480
Trainable params: 480
Non-trainable params: 0
_________________________________________________________________


In [11]:
i = "/Users/mgadgil/Documents/dog_breed_classification/data/train/0a001d75def0b4352ebde8d07c0850ae.jpg"
# show an image
# img = cv2.imread(i,0)
# plt.imshow(img)
# plt.show()

In [12]:
a = ResNet50_predict_labels(i)